In [2]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = '/kaggle/input/brain-tumor-mri-dataset'

In [4]:
!pip install scikit-learn
!pip install seaborn
!pip install plotly
!pip install missingno

In [3]:
import sys
import os
import math
import pathlib
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['figure.dpi'] = 300
%matplotlib inline
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import EfficientNetV2L

from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import *

from PIL import Image, ImageEnhance
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-07-01 01:54:58.656914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 01:54:58.657053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 01:54:58.787209: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_dir = pathlib.Path('/kaggle/input/brain-tumor-mri-dataset/Training')
test_dir = pathlib.Path('/kaggle/input/brain-tumor-mri-dataset/Testing')
img_height=224
img_width=224

In [5]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
])

# Normalization
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Dataset loading
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    interpolation="bilinear",
    label_mode="int",
    follow_links=False,
)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32,
)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 5712 files belonging to 4 classes.
Using 4570 files for training.
Found 5712 files belonging to 4 classes.
Using 1142 files for validation.
Found 1311 files belonging to 4 classes.


# Visualize the data distribution

In [6]:
ROOT_DIR = r"/kaggle/input/brain-tumor-mri-dataset"
TRAIN_DIR = os.path.join(ROOT_DIR, 'Training')
TEST_DIR = os.path.join(ROOT_DIR, 'Testing')
assert os.path.isdir(ROOT_DIR) and os.path.isdir(TRAIN_DIR) and os.path.isdir(TEST_DIR)
TUMOR_CLASS = ['meningioma', 'glioma', 'pituitary', 'notumor']
IMAGE_DATA_PATHS = [os.path.join(TRAIN_DIR, tumor_class) for tumor_class in TUMOR_CLASS]
TEST_DATA_PATHS = [os.path.join(TEST_DIR, tumor_class) for tumor_class in TUMOR_CLASS]

In [ ]:
TEST_DATA_PATHS

In [ ]:
data_distribution_count = pd.Series([len(os.listdir(path)) for path in TEST_DATA_PATHS if os.path.exists(path) and os.path.isdir(path)],index = TUMOR_CLASS)
data_distribution_count

In [ ]:
data_distribution_count = pd.Series([len(os.listdir(path)) for path in IMAGE_DATA_PATHS if os.path.exists(path) and os.path.isdir(path)],index = TUMOR_CLASS)
data_distribution_count

In [ ]:
def display_sample_images(paths, classes):
    fig, axes = plt.subplots(nrows=1, ncols=len(paths), figsize=(18, 5))
    for i, (path, cls) in enumerate(zip(paths, classes)):
        image_file = os.listdir(path)[1]
        image_path = os.path.join(path, image_file)
        image = plt.imread(image_path, format='grayscale')

        folder_name = os.path.basename(path)
        axes[i].imshow(image, cmap='gray')
        axes[i].set_title(folder_name)
        axes[i].axis('off')
    plt.show()


display_sample_images(IMAGE_DATA_PATHS, TUMOR_CLASS)

# Image Augmentation

# Inception V3

In [ ]:
base_model1 = InceptionV3(
                    input_shape=(224, 224, 3),
                    weights='imagenet',
                    include_top=False)

for layer in base_model1.layers[:10]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model1.output)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions1 = Dense(4, activation='softmax')(x)
model1 = Model(inputs=base_model1.inputs, outputs=predictions1)

model1.compile(optimizer=Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),
               loss=tf.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model1.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [ ]:
history1 = model1.fit(
    train_ds,
    shuffle=True,
    validation_data=test_ds,
    epochs=100,
    callbacks=[early_stop, model_checkpoint, reduce_lr]
)

model1.save('model1.keras')

In [ ]:
inceptionv3_evaluation = model1.evaluate(val_ds)
inceptionv3_evaluation[1]*100

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(acc))


plt.figure(figsize=(10, 5))
plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()


plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# VGG16

In [ ]:
base_model2 = VGG16(
                    input_shape=(224, 224, 3),
                    weights='imagenet',
                    include_top=False)

for layer in base_model2.layers[:10]:
    layer.trainable = False

x = base_model2.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions2 = Dense(4, activation='softmax')(x)
model2 = Model(inputs=base_model2.inputs, outputs=predictions2)

model2.compile(optimizer=Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),
               loss=tf.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model2.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [ ]:
history2 = model2.fit(
    train_ds,
    shuffle=True,
    validation_data=test_ds,
    epochs=100,
    callbacks=[early_stop, model_checkpoint, reduce_lr]
)

model2.save('model2.keras')

In [ ]:
vgg16_evaluation = model2.evaluate(val_ds)
vgg16_evaluation[1]*100

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(acc))
plt.figure(figsize=(10, 5))
plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# ResNet152V2

In [ ]:
base_model3 = ResNet152V2(
                    input_shape=(224, 224, 3),
                    weights='imagenet',
                    include_top=False)


for layer in base_model3.layers[:10]:
    layer.trainable = False

x = base_model3.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions3 = Dense(4, activation='softmax')(x)
model3 = Model(inputs=base_model3.inputs, outputs=predictions3)

model3.compile(optimizer=Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),
               loss=tf.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model3.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [ ]:
history3=model3.fit(
  train_ds,
  shuffle=True,
  validation_data=test_ds,
  epochs=100,
  callbacks=[early_stop, model_checkpoint, reduce_lr])

model3.save('model3.keras')

In [ ]:
resnet152v2_evaluation = model3.evaluate(val_ds)
resnet152v2_evaluation[1]*100

In [ ]:
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(len(acc))
plt.figure(figsize=(10, 5))
plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# MobileNetV2

In [ ]:
base_model4 = MobileNetV2(
                    input_shape=(224, 224, 3),
                    weights='imagenet',
                    include_top=False)

for layer in base_model4.layers[:10]:
    layer.trainable = False

x = base_model4.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions4 = Dense(4, activation='softmax')(x)
model4 = Model(inputs=base_model4.inputs, outputs=predictions4)

model4.compile(optimizer=Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),
               loss=tf.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model4.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [ ]:
history4=model4.fit(
  train_ds,
  shuffle=True,
  validation_data=test_ds,
  epochs=100,
  callbacks=[early_stop, model_checkpoint, reduce_lr])

model4.save('model4.keras')

In [ ]:
mobilenet_evaluation = model4.evaluate(val_ds)
mobilenet_evaluation[1]*100

In [ ]:
acc = history4.history['accuracy']
val_acc = history4.history['val_accuracy']
loss = history4.history['loss']
val_loss = history4.history['val_loss']

epochs = range(len(acc))
plt.figure(figsize=(10, 5))
plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Xception

In [ ]:
base_model5 = Xception(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)

for layer in base_model5.layers[:10]:
    layer.trainable = False
x = base_model5.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions5 = Dense(4, activation='softmax')(x)
model5 = Model(inputs=base_model5.inputs, outputs=predictions5)

model5.compile(optimizer=Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),
               loss=tf.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model5.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [ ]:
history5=model5.fit(
  train_ds,
  shuffle=True,
  validation_data=test_ds,
  epochs=100,
  callbacks=[early_stop, model_checkpoint, reduce_lr])

model5.save('model5.keras')

In [ ]:
xception_evaluation = model5.evaluate(val_ds)
xception_evaluation[1]*100

In [ ]:
acc = history5.history['accuracy']
val_acc = history5.history['val_accuracy']
loss = history5.history['loss']
val_loss = history5.history['val_loss']

epochs = range(len(acc))
plt.figure(figsize=(10, 5))
plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# EfficientNetV2L

In [ ]:
base_model6 = EfficientNetV2L(
                        input_shape=(224, 224, 3),
                        weights='imagenet',
                        include_top=False)

for layer in base_model6.layers[:10]:
    layer.trainable = False
x = base_model6.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.4)(x)
predictions6 = Dense(4, activation='softmax')(x)
model6 = Model(inputs=base_model6.inputs, outputs=predictions6)

model6.compile(optimizer=Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999),
               loss=tf.losses.SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model6.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [ ]:
history6=model6.fit(
  train_ds,
  shuffle=True,
  validation_data=test_ds,
  epochs=100,
  callbacks=[early_stop, model_checkpoint, reduce_lr])

model6.save('model6.keras')

In [ ]:
efficientnet_evaluation = model6.evaluate(val_ds)
efficientnet_evaluation[1]*100

In [ ]:
acc = history6.history['accuracy']
val_acc = history6.history['val_accuracy']
loss = history6.history['loss']
val_loss = history6.history['val_loss']

epochs = range(len(acc))
plt.figure(figsize=(10, 5))
plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()